In [1]:
import numpy as np
import pandas as pd

In [2]:
import torch
import sklearn
import random

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [4]:
torch.manual_seed(1)
random.seed(1)
if device == "cuda":
    torch.cuda.manual_seed(1)

In [5]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self,x,y):
        self.x = torch.Tensor(x).to(device)
        self.y = torch.Tensor(y).to(device)
    def __getitem__(self,index):
        return self.x[index],self.y[index]
    def __len__(self):
        return len(self.x)

In [6]:
x_train = pd.read_csv("./data2/x_train.csv",engine='python',encoding='CP949')
y_train = pd.read_csv("./data2/y_train.csv",engine='python',encoding='CP949')
x_test = pd.read_csv("./data2/x_test.csv",engine='python',encoding='CP949')
submission = pd.read_csv("./data2/sample_submission.csv")

In [7]:
print(x_train)
print(y_train)
print(x_test)
print(submission)

       시도명  읍면동명         거주인구        근무인구        방문인구       총 유동인구   평균 속도  \
0        0    26    76018.965    5009.811   36887.341   117916.117  40.467   
1        1    24  1232416.968   76043.955  451558.268  1760019.191  38.126   
2        1    23   134260.946    8412.761   78177.981   220851.688  51.107   
3        1    25   289327.429   39102.424  287029.303   615459.156  33.067   
4        1     7  1116617.660   96560.651  524905.301  1738083.612  40.392   
...    ...   ...          ...         ...         ...          ...     ...   
21481    0     6    91977.598   14802.819   79885.446   186665.863  41.676   
21482    0     9    63673.610    3725.663   65813.385   133212.658  48.737   
21483    1     3   406035.642   46335.550  258079.137   710450.329  49.314   
21484    1    25   315737.894   25379.224  283079.548   624196.666  35.312   
21485    1    27  1092141.759  185878.376  518636.283  1796656.419  33.438   

       평균 소요 시간   평균 기온    일강수량  평균 풍속   월   일     년  
0       

In [8]:
x_train = np.array(x_train)
y_train = np.array(y_train) # shape (21486,1)
# y_train = np.array(y_train['교통량']) # shape (21486)
x_test = np.array(x_test)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [9]:
train_dataset = MyDataset(x_train,y_train)
x_train = torch.Tensor(x_train).to(device)
y_train = torch.Tensor(y_train).to(device)
x_test = torch.Tensor(x_test).to(device)
print(x_train.shape,y_train.shape)

torch.Size([21486, 14]) torch.Size([21486, 1])


In [10]:
layer1 = torch.nn.Linear(14,64,bias=True).to(device)
layer2 = torch.nn.Linear(64,64,bias=True).to(device)
layer3 = torch.nn.Linear(64,1,bias=True).to(device)
relu = torch.nn.ReLU().to(device)
dropout = torch.nn.Dropout(0.3).to(device)

In [11]:
torch.nn.init.xavier_normal_(layer1.weight)
torch.nn.init.xavier_normal_(layer2.weight)
torch.nn.init.xavier_normal_(layer3.weight)


Parameter containing:
tensor([[-1.0191e-01, -1.7921e-01,  1.4614e-01,  5.6814e-02,  8.3614e-02,
          1.3077e-01, -1.0220e-01,  3.2455e-01,  3.5776e-02,  4.2419e-02,
          8.5779e-02, -1.5427e-01,  1.0593e-01, -1.0059e-01, -2.1393e-01,
         -3.2475e-01,  1.4998e-01,  4.2955e-02, -2.5857e-02,  2.5351e-01,
         -1.4957e-01,  1.6412e-01, -1.7423e-01, -1.2037e-01, -1.1412e-01,
          9.9023e-02,  1.0749e-01,  3.4918e-01, -1.1516e-02, -3.7527e-01,
         -2.3346e-02, -1.2157e-01,  1.4791e-01,  4.3880e-02,  3.4508e-01,
         -1.2737e-01, -2.1519e-01,  3.4951e-02, -2.5725e-01, -1.1786e-01,
         -1.9877e-01,  2.0981e-01, -3.8434e-03, -4.9136e-03, -2.0228e-02,
          2.6726e-04,  1.3832e-01,  5.1857e-02,  1.9495e-01,  1.6591e-01,
         -1.4029e-01,  1.3367e-01,  1.7965e-01, -2.9983e-01, -5.9664e-02,
         -5.4871e-02,  2.3979e-01, -1.3352e-01, -3.2272e-02, -1.3104e-01,
          1.1347e-02,  2.2168e-01,  7.3924e-02,  5.0304e-02]], device='cuda:0',
       req

In [12]:
model = torch.nn.Sequential(layer1,relu,dropout,
                            layer2,relu,dropout,
                            layer3).to(device)

In [13]:
def r2_loss(output, target):
    target_mean = torch.mean(target)
    ss_tot = torch.sum((target - target_mean) ** 2)
    ss_res = torch.sum((target - output) ** 2)
    r2 = 1 - ss_res / ss_tot
    return r2

In [14]:
# layer 5, epochs 2000, lr 1e-2 -> score 0.474...
# layer 5, epochs 2000, lr 1e-1 -> score 0.385... overfitting
# layer 7, epochs 2000, lr 1e-2 -> cost 105335.1015625 score 0.11277 
# batch size = 100 layer 7, epochs 1000, lr 1e-2 -> 하나의 예측값
# batch size = 100 layer 7, epochs 1000, lr 1e-2 -> 하나의 예측값 
epochs= 500
lr = 2e-2
loss = torch.nn.MSELoss().to(device)

optim = torch.optim.Adam(model.parameters(),lr=lr)
batch_size = 512
data_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                         batch_size = batch_size,
                                         shuffle=True,
                                         drop_last =True)

In [15]:
model.train()
for epoch in range(epochs+1):
    avg_cost = 0
    for x,y in data_loader:
        optim.zero_grad()
    
        h = model(x) 
        cost = loss(h,y)
        cost.backward()
        optim.step()
        avg_cost += cost.item()
    if epoch % 10 == 0:
        print(epoch, avg_cost/len(data_loader))

0 270877.3551829268
10 127286.27324695123
20 69808.16539634146
30 58885.48904344512
40 49080.119950457316
50 46827.03572789634
60 42797.81440548781
70 40475.13862423781
80 39017.99657012195
90 37246.61652057927
100 35369.239615091465
110 34866.30125762195
120 33733.67439977134
130 33659.032583841465
140 32167.446836890245
150 32343.73799542683
160 30606.028582317074
170 30068.658060213416
180 29821.01143292683
190 29834.551924542684
200 29698.594369283535
210 29716.190167682926
220 28480.980802210364
230 28819.033250762193
240 28359.30078125
250 28244.567120807926
260 28069.317406631097
270 27303.817216082316
280 27428.015005716465
290 27538.385718368903
300 27630.69745617378
310 26428.003096417684
320 26944.990758384145
330 26646.922541920732
340 27018.327315167684
350 25894.47022675305
360 27191.520817454268
370 25498.724180640245
380 27038.999857088416
390 26035.167444740855
400 26041.158631859755
410 24900.18907202744
420 25760.838986280487
430 25612.233469893294
440 25569.90463033

In [16]:
from sklearn.metrics import r2_score
with torch.no_grad():
    model.eval()
    predict = model(x_train)
    
print("[R2_Loss] =",r2_loss(predict,y_train))

[R2_Loss] = tensor(0.9538, device='cuda:0')


In [17]:
with torch.no_grad():
    model.eval()
    predict = model(x_test)
submission['predict'] = predict.cpu().detach().numpy()
print(submission)

          id      predict
0          0    37.778427
1          1    54.567867
2          2    24.011959
3          3   959.295776
4          4  1731.441162
...      ...          ...
10578  10578    19.936024
10579  10579    14.203346
10580  10580    54.044769
10581  10581    26.102608
10582  10582   717.497925

[10583 rows x 2 columns]


In [18]:
submission.to_csv("submission.csv",index=False)

In [ ]:
# 저번 과제와 같이 한가지 예측값으로 통일되는 경향이 생김 -> batch size?
# batch size를 적용해도 계속 같은 문제 -> lr 조정
# lr를 적용해도 계속 같은 문제 -> drop late 조정

## 결론 -> Hidden Layer의 노드 수는 컬럼의 수와 관계가 없다. 컬럼의 수보다 많아도 되고 적어도 된다. 
# 학습이 제대로 되지 않을때 노드의 수를 늘려보는 방법도 좋다 (단, 많을 수록 overfit 문제 가능성 + 학습이 안될 가능성이 있음.)

# layer 5개 -> 학습은 되지만 baseline을 넘지 못함
# layer 7 -> 5개 보다 늦은 점수를 받음
# layer 3 -> 더 좋은 성능을 보인다. --> layer가 많다고 좋은건 아님 (Gradient Vanishing)